### data load

In [32]:
import pandas as pd
# 파일 이름/경로는 환경에 맞게
df = pd.read_csv("C:/Users/Admin/Desktop/PROJ/data/01_raw/fra_cleaned.csv", encoding="cp1252", sep=";")

### gender & brand 필터링

In [ ]:
target_brands = [
    "diptyque",
    "lanvin",
    "byredo",
    "yvessaintlaurent",
    "santamarianovella",
    "lelabo",
    "maisonfranciskurkdjian",
    "chanel",
    "acquadiparma",
    "jomalonelondon",
    "jimmychoo",
    "guerlain",
    "calvinklein",
    "arianagrande",
    "versace",
    "exnihilo",
    "memoparis",
    "demeterfragrance",
    "elizabetharden",
    "creed",
    "giorgioarmani",
]

# 브랜드 정리
def is_target_brand(brand: str) -> bool:
    if pd.isna(brand):
        return False
    norm = str(brand).replace("-", "").lower()
    return any(t in norm for t in target_brands)

mask = df["Brand"].apply(is_target_brand)

df_filtered = df[mask].copy()

# 성별 정리
filtered_df = df_filtered[df_filtered["Gender"].str.lower().isin(["women", "unisex"])]

In [ ]:
# 향수 분석에 필요없는 컬럼 제거
filtered_df = filtered_df.drop(columns=['Country','Year','Perfumer1','Perfumer2'], axis=1)

#### perfume_id(PK) 추가

In [ ]:
# 인덱스 정렬하고 인덱스를 컬럼으로 추가(PK)
df_final = filtered_df.reset_index(drop=True)
df_final = df_final.reset_index().rename(columns={"index": "perfume_id"})

In [ ]:
# df_final.to_csv("C:/Users/Admin/Desktop/PROJ/data/02_cleaned/0.perfume_gender_brand_filtered.csv",
#     index=False,
#     encoding="utf-8-sig"
# )

### (향수 id & url만 파일 생성)

In [53]:
# 향수 id & url만 
df_id = df_final.iloc[:,:2]

In [ ]:
# df_id.to_csv("C:/Users/Admin/Desktop/PROJ/data/02_cleaned/perfume_id_url.csv",
#     index=False,
#     encoding="utf-8-sig"
# )

### mainacccord 결측치 처리

36개의 행 추가 크롤링으로 처리

### mainaccord 이상치 제거 & 한정판 향수 제거 

In [ ]:
file_path = 'C:/Users/Admin/Desktop/PROJ/data/02_cleaned/perfume/1.perfume_mainaccord_missing_filled.csv'
df_clean = pd.read_csv(file_path)

accord_cols = ['mainaccord1', 'mainaccord2', 'mainaccord3', 'mainaccord4', 'mainaccord5']

# 소문자로 변환 & 앞뒤 공백 제거
for col in accord_cols:
    df_clean[col] = df_clean[col].astype(str).str.lower().str.strip()
    
#빈도수 계산
all_accords = df_clean[accord_cols].stack()
accord_counts = all_accords.value_counts()

# 기준: 전체에서 20번 미만 등장하면 삭제
threshold = 20

# 기준 미만인 '희귀 향' 리스트 추출
rare_accords = set(accord_counts[accord_counts < threshold].index)

# 희귀 향이 하나라도 포함된 행 찾기
condition_mask = df_clean[accord_cols].isin(rare_accords).any(axis=1)

# 희귀 향이 '없는' 행만 남기기 (~)
df_final = df_clean[~condition_mask]

#limited-edition 포합된 행 지우기
filtered_df = df_final[~df_final['Perfume'].str.contains('limited-edition', case=False, na=False)]

# #저장
# filtered_df.to_csv('C:/Users/Admin/Desktop/PROJ/data/02_cleaned/perfume/2.perfume_limited_mainaccord_filtered.csv', index=False)